In [4]:
import torch
import torch.nn.functional as F
import numpy as np
print(torch.__version__)
print(np.__version__)

1.4.0
1.17.2


In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


In [6]:
var = torch.tensor([[10,3], [2,8], [10,2], [3,7]], dtype=torch.float32)
print(var)
n,d = var.size()
print('n=', n, ', d=', d)

print('torch.sigmoid(tensor):')
print(torch.sigmoid(var))

print('F.log_softmax(tensor, dim=1):')
print(F.log_softmax(var, dim=1))

print('F.log_softmax(torch.sigmoid(tensor), dim=1):')
print(F.log_softmax(torch.sigmoid(var), dim=1))

tensor([[10.,  3.],
        [ 2.,  8.],
        [10.,  2.],
        [ 3.,  7.]])
n= 4 , d= 2
torch.sigmoid(tensor):
tensor([[1.0000, 0.9526],
        [0.8808, 0.9997],
        [1.0000, 0.8808],
        [0.9526, 0.9991]])
F.log_softmax(tensor, dim=1):
tensor([[-9.1142e-04, -7.0009e+00],
        [-6.0025e+00, -2.4757e-03],
        [-3.3540e-04, -8.0003e+00],
        [-4.0181e+00, -1.8150e-02]])
F.log_softmax(torch.sigmoid(tensor), dim=1):
tensor([[-0.6697, -0.7171],
        [-0.7543, -0.6355],
        [-0.6353, -0.7545],
        [-0.7167, -0.6702]])


In [12]:
def loss_crossentropy(bottleneck, reduction, f):
    loss_fun = torch.nn.CrossEntropyLoss(reduction=reduction)
    bt = torch.sigmoid(bottleneck).to(device)  # sigmoid because we need the probability distributions   
    _, preds = torch.max(bt, 1)
    if f==1:
        loss_ret_1 = loss_fun(bottleneck.to(device), preds)    
    elif f==2:
        loss_ret_1 = loss_fun(F.log_softmax(bottleneck, dim=1).to(device), preds)
    else:
        loss_ret_1 = loss_fun(F.log_softmax(bt, dim=1).to(device), preds)
    return loss_ret_1

In [13]:
print('loss_crossentropy-1-mean')
print(loss_crossentropy(var, reduction='mean', f=1))

loss_crossentropy-1-mean
tensor(0.0055, device='cuda:0')


In [14]:
print('loss_crossentropy-2-mean')
print(loss_crossentropy(var, reduction='mean', f=2))

loss_crossentropy-2-mean
tensor(0.0055, device='cuda:0')


In [15]:
print('loss_crossentropy-3-mean')
print(loss_crossentropy(var, reduction='mean', f=3))

loss_crossentropy-3-mean
tensor(0.6527, device='cuda:0')


In [16]:
print('loss_crossentropy-1-sum')
print(loss_crossentropy(var, reduction='sum', f=1))

loss_crossentropy-1-sum
tensor(0.0219, device='cuda:0')


In [ ]:
def loss_crossentropy(bottleneck, reduction):
    loss_fun = torch.nn.CrossEntropyLoss(reduction=reduction)
    bt = torch.sigmoid(bottleneck).to(device)  # sigmoid because we need the probability distributions   
    _, preds = torch.max(bt, 1)
    loss_ret_1 = loss_fun(F.log_softmax(bt, dim=1).to(device), preds)
    return loss_ret_1